In [ ]:
from data_reader_writer import *
from cluster import *

import numpy as np
from sklearn.cluster import SpectralBiclustering, SpectralCoclustering
from scipy.stats import spearmanr

from math import ceil
import pickle

In [ ]:
all_datasets =  pickle.load(open("all_datasets_2.p", "rb")) 

In [ ]:
tw_per_els = [1, 2, 3]
results = {}

for s in SUBJECTS:
    print(s)
    for tw_type in time_window_types:
        print(tw_type)
        for trial_type in trial_types:
            print(trial_type)
            for tw_per_el in tw_per_els:
                for use_all_els in [True, False]:
                    data = all_datasets[(s, tw_type, trial_type, tw_per_el, use_all_els)]
                    
                    max_clusters = None 
                    if min(data.shape) <= 10:
                        max_clusters = ceil(min(data.shape) / 2) 
                    elif min(data.shape) <= 20:
                        max_clusters = ceil(min(data.shape) / 3) 
                    else:
                        max_clusters = 8
                    
                    for n_clusters in range(2, max_clusters + 1):
                        for svd_method in ["randomized", "arpack"]:
                            try:
                                row_labels, col_labels, rows, cols = bicocluster(data, n_clusters=n_clusters, svd_method=svd_method, 
                                                                         plot_results=False)
                                
                                results[(s, tw_type, trial_type, tw_per_el, use_all_els, n_clusters, 
                                         svd_method)] = (ASR(data, row_labels, col_labels), row_labels, col_labels, rows, cols)
                            except Exception as e:
                                print("Failed with params: ", s, tw_type, trial_type, tw_per_el, use_all_els, n_clusters, 
                                      svd_method)
                                print("Trace: ", str(e))
                                

In [ ]:
pickle.dump(results, open("cocluster_results_3.p", "wb"))